Team 15 : Ablation : Extension of G-BERT by replacing BERT with GPT2

In [3]:
# Importing libraries 
 
import torch  

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import LayerNorm
from torch.nn import CrossEntropyLoss, BCEWithLogitsLoss  
import torch.nn.functional as F 
import numpy as np

import inspect

from torch_geometric.utils import scatter_, softmax, add_self_loops
# Update for torch_geometric latest version 
#from torch_geometric.utils import scatter, softmax, add_self_loops
from torch_geometric.nn.inits import glorot, zeros, uniform

from build_tree import build_stage_one_edges, build_stage_two_edges, build_cominbed_edges
from build_tree import build_icd9_tree, build_atc_tree

import json  
import copy  


 
#from transformers import GPT2Config, GPT2Model  
from transformers import GPT2Model  
from transformers.models.gpt2.modeling_gpt2 import GPT2Model 
import logging  
import os
import pandas as pd
#from graph_models import FuseEmbeddings
  
logger = logging.getLogger(__name__)  
  
CONFIG_NAME = 'gpt2_config.json'  
WEIGHTS_NAME = 'pytorch_model2.bin'
from tqdm import tqdm, trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset
from torch.utils.data.distributed import DistributedSampler
from torch.optim import Adam


Update for Graph Models to fit GPT2 Architecture and config.

In [11]:
# Used as is from Original Code

class MessagePassing(nn.Module):
    r"""Base class for creating message passing layers

    .. math::
        \mathbf{x}_i^{\prime} = \gamma_{\mathbf{\Theta}} \left( \mathbf{x}_i,
        \square_{j \in \mathcal{N}(i)} \, \phi_{\mathbf{\Theta}}
        \left(\mathbf{x}_i, \mathbf{x}_j,\mathbf{e}_{i,j}\right) \right),

    where :math:`\square` denotes a differentiable, permutation invariant
    function, *e.g.*, sum, mean or max, and :math:`\gamma_{\mathbf{\Theta}}`
    and :math:`\phi_{\mathbf{\Theta}}` denote differentiable functions such as
    MLPs.
    See `here <https://rusty1s.github.io/pytorch_geometric/build/html/notes/
    create_gnn.html>`__ for the accompanying tutorial.

    """

    def __init__(self, aggr='add'):
        super(MessagePassing, self).__init__()

        self.message_args = inspect.getargspec(self.message)[0][1:]
        self.update_args = inspect.getargspec(self.update)[0][2:]

    def propagate(self, aggr, edge_index, **kwargs):
        r"""The initial call to start propagating messages.
        Takes in an aggregation scheme (:obj:`"add"`, :obj:`"mean"` or
        :obj:`"max"`), the edge indices, and all additional data which is
        needed to construct messages and to update node embeddings."""

        assert aggr in ['add', 'mean', 'max']
        kwargs['edge_index'] = edge_index

        size = None
        message_args = []
        for arg in self.message_args:
            if arg[-2:] == '_i':
                tmp = kwargs[arg[:-2]]
                size = tmp.size(0)
                message_args.append(tmp[edge_index[0]])
            elif arg[-2:] == '_j':
                tmp = kwargs[arg[:-2]]
                size = tmp.size(0)
                message_args.append(tmp[edge_index[1]])
            else:
                message_args.append(kwargs[arg])

        update_args = [kwargs[arg] for arg in self.update_args]

        out = self.message(*message_args)
        out = scatter_(aggr, out, edge_index[0], dim_size=size)
        # update for latest torch_geometric
#        out = scatter(aggr, out, edge_index[0], dim_size=size, reduce="add" )
        out = self.update(out, *update_args)

        return out

    def message(self, x_j):  # pragma: no cover
        r"""Constructs messages in analogy to :math:`\phi_{\mathbf{\Theta}}`
        for each edge in :math:`(i,j) \in \mathcal{E}`.
        Can take any argument which was initially passed to :meth:`propagate`.
        In addition, features can be lifted to the source node :math:`i` and
        target node :math:`j` by appending :obj:`_i` or :obj:`_j` to the
        variable name, *.e.g.* :obj:`x_i` and :obj:`x_j`."""

        return x_j

    def update(self, aggr_out):  # pragma: no cover
        r"""Updates node embeddings in analogy to
        :math:`\gamma_{\mathbf{\Theta}}` for each node
        :math:`i \in \mathcal{V}`.
        Takes in the output of aggregation as first argument and any argument
        which was initially passed to :meth:`propagate`."""

        return aggr_out

In [12]:
# Used as is from Original Code
class GATConv(MessagePassing):
    r"""The graph attentional operator from the `"Graph Attention Networks"
    <https://arxiv.org/abs/1710.10903>`_ paper

    .. math::
        \mathbf{x}^{\prime}_i = \alpha_{i,i}\mathbf{\Theta}\mathbf{x}_{j} +
        \sum_{j \in \mathcal{N}(i)} \alpha_{i,j}\mathbf{\Theta}\mathbf{x}_{j},

    where the attention coefficients :math:`\alpha_{i,j}` are computed as

    .. math::
        \alpha_{i,j} =
        \frac{
        \exp\left(\mathrm{LeakyReLU}\left(\mathbf{a}^{\top}
        [\mathbf{\Theta}\mathbf{x}_i \, \Vert \, \mathbf{\Theta}\mathbf{x}_j]
        \right)\right)}
        {\sum_{k \in \mathcal{N}(i) \cup \{ i \}}
        \exp\left(\mathrm{LeakyReLU}\left(\mathbf{a}^{\top}
        [\mathbf{\Theta}\mathbf{x}_i \, \Vert \, \mathbf{\Theta}\mathbf{x}_k]
        \right)\right)}.

    Args:
        in_channels (int): Size of each input sample.
        out_channels (int): Size of each output sample.
        heads (int, optional): Number of multi-head-attentions. (default:
            :obj:`1`)
        concat (bool, optional): If set to :obj:`False`, the multi-head
        attentions are averaged instead of concatenated. (default: :obj:`True`)
        negative_slope (float, optional): LeakyReLU angle of the negative
            slope. (default: :obj:`0.2`)
        dropout (float, optional): Dropout probability of the normalized
            attention coefficients which exposes each node to a stochastically
            sampled neighborhood during training. (default: :obj:`0`)
        bias (bool, optional): If set to :obj:`False`, the layer will not learn
            an additive bias. (default: :obj:`True`)
    """

    def __init__(self,
                 in_channels,
                 out_channels,
                 heads=1,
                 concat=True,
                 negative_slope=0.2,
                 dropout=0,
                 bias=True):
        super(GATConv, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = heads
        self.concat = concat
        self.negative_slope = negative_slope
        self.dropout = dropout

        self.weight = nn.Parameter(
            torch.Tensor(in_channels, heads * out_channels))
        self.att = nn.Parameter(torch.Tensor(1, heads, 2 * out_channels))

        if bias and concat:
            self.bias = nn.Parameter(torch.Tensor(heads * out_channels))
        elif bias and not concat:
            self.bias = nn.Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        glorot(self.weight)
        glorot(self.att)
        zeros(self.bias)

    def forward(self, x, edge_index):
        """"""
        edge_index = add_self_loops(edge_index, num_nodes=x.size(0))
        x = torch.mm(x, self.weight).view(-1, self.heads, self.out_channels)
        return self.propagate('add', edge_index, x=x, num_nodes=x.size(0))

    def message(self, x_i, x_j, edge_index, num_nodes):
        # Compute attention coefficients.
        alpha = (torch.cat([x_i, x_j], dim=-1) * self.att).sum(dim=-1)
        alpha = F.leaky_relu(alpha, self.negative_slope)
        alpha = softmax(alpha, edge_index[0], num_nodes)

        alpha = F.dropout(alpha, p=self.dropout)

        return x_j * alpha.view(-1, self.heads, 1)

    def update(self, aggr_out):
        if self.concat is True:
            aggr_out = aggr_out.view(-1, self.heads * self.out_channels)
        else:
            aggr_out = aggr_out.mean(dim=1)

        if self.bias is not None:
            aggr_out = aggr_out + self.bias
        return aggr_out

    def __repr__(self):
        return '{}({}, {}, heads={})'.format(self.__class__.__name__,
                                             self.in_channels,
                                             self.out_channels, self.heads)

In [13]:
# Used as is from Original Code

class OntologyEmbedding(nn.Module):
    def __init__(self, voc, build_tree_func,
                 in_channels=100, out_channels=20, heads=5):
        super(OntologyEmbedding, self).__init__()

        # initial tree edges
        res, graph_voc = build_tree_func(list(voc.idx2word.values()))
        stage_one_edges = build_stage_one_edges(res, graph_voc)
        stage_two_edges = build_stage_two_edges(res, graph_voc)

        self.edges1 = torch.tensor(stage_one_edges)
        self.edges2 = torch.tensor(stage_two_edges)
        self.graph_voc = graph_voc

        # construct model
        assert in_channels == heads * out_channels
        self.g = GATConv(in_channels=in_channels,
                         out_channels=out_channels,
                         heads=heads)

        # tree embedding
        num_nodes = len(graph_voc.word2idx)
        self.embedding = nn.Parameter(torch.Tensor(num_nodes, in_channels))

        # idx mapping: FROM leaf node in graphvoc TO voc
        self.idx_mapping = [self.graph_voc.word2idx[word]
                            for word in voc.idx2word.values()]

        self.init_params()

    def get_all_graph_emb(self):
        emb = self.embedding
        emb = self.g(self.g(emb, self.edges1.to(emb.device)),
                     self.edges2.to(emb.device))
        return emb

    def forward(self):
        """
        :param idxs: [N, L]
        :return:
        """
        emb = self.embedding

        emb = self.g(self.g(emb, self.edges1.to(emb.device)),
                     self.edges2.to(emb.device))

        return emb[self.idx_mapping]

    def init_params(self):
        glorot(self.embedding)

In [14]:
# Updates to config.xxxxx parameters. Original Config parameter refered to BERT Architecture
# New Config params refer to GPT2 Arch hidden_size --> n.embd 

class ConcatEmbeddings(nn.Module):
    """Concat rx and dx ontology embedding for easy access
    """

    def __init__(self, config, dx_voc, rx_voc):
        super(ConcatEmbeddings, self).__init__()
        # special token: "[PAD]", "[CLS]", "[MASK]"
        # special token: "[PAD]", "[CLS]", "[EOS]" --> GPT2
        # self.special_embedding = nn.Parameter(
        #     torch.Tensor(config.vocab_size - len(dx_voc.idx2word) - len(rx_voc.idx2word), config.hidden_size))
        # self.rx_embedding = OntologyEmbedding(rx_voc, build_atc_tree,
        #                                       config.hidden_size, config.graph_hidden_size,
        #                                       config.graph_heads)
        # self.dx_embedding = OntologyEmbedding(dx_voc, build_icd9_tree,
        #                                       config.hidden_size, config.graph_hidden_size,
        #                                       config.graph_heads)
        self.special_embedding = nn.Parameter(
            torch.Tensor(config.vocab_size - len(dx_voc.idx2word) - len(rx_voc.idx2word), config.n_embd))
        self.rx_embedding = OntologyEmbedding(rx_voc, build_atc_tree,
                                              config.n_embd, config.graph_hidden_size,
                                              config.graph_heads)
        self.dx_embedding = OntologyEmbedding(dx_voc, build_icd9_tree,
                                              config.n_embd, config.graph_hidden_size,
                                              config.graph_heads)
        self.init_params()

    def forward(self, input_ids):
        emb = torch.cat(
            [self.special_embedding, self.rx_embedding(), self.dx_embedding()], dim=0)
        return emb[input_ids]

    def init_params(self):
        glorot(self.special_embedding)

In [15]:
# Updates to config.xxxxx parameters. Original Config parameter refered to BERT Architecture
# New Config params refer to GPT2 Arch hidden_size --> n.embd 

class FuseEmbeddings(nn.Module):
    """Construct the embeddings from ontology, patient info and type embeddings.
    """

    def __init__(self, config, dx_voc, rx_voc):
        super(FuseEmbeddings, self).__init__()
        self.ontology_embedding = ConcatEmbeddings(config, dx_voc, rx_voc)
        # self.type_embedding = nn.Embedding(2, config.hidden_size)
        self.type_embedding = nn.Embedding(2, config.n_embd)
        # self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)

    def forward(self, input_ids, input_types=None, input_positions=None):
        """
        :param input_ids: [B, L]
        :param input_types: [B, L]
        :param input_positions:
        :return:
        """
        # return self.ontology_embedding(input_ids)
        ontology_embedding = self.ontology_embedding(
            input_ids) + self.type_embedding(input_types)
        return ontology_embedding

Attempt to Replace BERT with GPT2

Adapting BERTConfig from config.py to GPT2Config

In [16]:
class GPT2Config(object):  
    def __init__(self,  
                 vocab_size_or_config_json_file,  
                 n_ctx=1024,  
                 n_embd=300,  
                 n_head=12,  
                 n_layer=2,  
                 initializer_range=0.02,
                 graph=False,
                 graph_hidden_size=75,
                 graph_heads=4):  
        if isinstance(vocab_size_or_config_json_file, str):  
            with open(vocab_size_or_config_json_file, "r", encoding='utf-8') as reader:  
                json_config = json.loads(reader.read())  
            for key, value in json_config.items():  
                self.__dict__[key] = value  
        elif isinstance(vocab_size_or_config_json_file, int):  
            self.vocab_size = vocab_size_or_config_json_file  
            self.n_ctx = n_ctx  
            self.n_embd = n_embd  
            self.n_head = n_head  
            self.n_layer = n_layer  
            self.initializer_range = initializer_range 
            self.graph = graph
            self.graph_hidden_size = graph_hidden_size
            self.graph_heads = graph_heads 
        else:  
            raise ValueError("First argument must be either a vocabulary size (int)"  
                             "or the path to a pretrained model config file (str)")  
  
    @classmethod  
    def from_dict(cls, json_object):  
        config = GPT2Config(vocab_size_or_config_json_file=-1)  
        for key, value in json_object.items():  
            config.__dict__[key] = value  
        return config  
  
    @classmethod  
    def from_json_file(cls, json_file):  
        with open(json_file, "r", encoding='utf-8') as reader:  
            text = reader.read()  
        return cls.from_dict(json.loads(text))  
  
    def __repr__(self):  
        return str(self.to_json_string())  
  
    def to_dict(self):  
        output = copy.deepcopy(self.__dict__)  
        return output  
  
    def to_json_string(self):  
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n" 

PreTrainedBertModel to PreTrainedGPT2Model

In [17]:
class PreTrainedGPT2Model(nn.Module):  
    """ An abstract class to handle weights initialization and  
        a simple interface for downloading and loading pretrained models.  
    """  
  
    def __init__(self, config: GPT2Config, *inputs, **kwargs):  
        super(PreTrainedGPT2Model, self).__init__()  
        if not isinstance(config, GPT2Config):  
            raise ValueError(  
                "Parameter config in `{}(config)` should be an instance of class `GPT2Config`. "  
                "To create a model from a pretrained model use "  
                "`model = {}.from_pretrained(PRETRAINED_MODEL_NAME)`".format(  
                    self.__class__.__name__, self.__class__.__name__  
                ))  
        self.config = config  
  
    def init_gpt2_weights(self, module):  
        """ Initialize the weights.  
        """  
        if isinstance(module, (nn.Linear, nn.Embedding)):  
            module.weight.data.normal_(  
                mean=0.0, std=self.config.initializer_range)  
        elif isinstance(module, nn.LayerNorm):  
            module.bias.data.zero_()  
            module.weight.data.fill_(1.0)  
        if isinstance(module, nn.Linear) and module.bias is not None:  
            module.bias.data.zero_()  
  
    @classmethod  
    def from_pretrained(cls, pretrained_model_name, state_dict=None, cache_dir='', *inputs, **kwargs):  
        serialization_dir = os.path.join(cache_dir, pretrained_model_name)  
        # Load config  
        config_file = os.path.join(serialization_dir, CONFIG_NAME)  
        config = GPT2Config.from_json_file(config_file)  
        logger.info("Model config {}".format(config))  
        # Instantiate model.  
        model = cls(config, *inputs, **kwargs)  
        if state_dict is None:  
            weights_path = os.path.join(serialization_dir, WEIGHTS_NAME)  
            state_dict = torch.load(weights_path, map_location=torch.device('cpu'))  
  
        missing_keys = []  
        unexpected_keys = []  
        error_msgs = []  
        # copy state_dict so _load_from_state_dict can modify it  
        metadata = getattr(state_dict, '_metadata', None)  
        state_dict = state_dict.copy()  
        if metadata is not None:  
            state_dict._metadata = metadata  
  
        def load(module, prefix=''):  
            local_metadata = {} if metadata is None else metadata.get(  
                prefix[:-1], {})  
            module._load_from_state_dict(  
                state_dict, prefix, local_metadata, True, missing_keys, unexpected_keys, error_msgs)  
            for name, child in module._modules.items():  
                if child is not None:  
                    load(child, prefix + name + '.')  
  
        load(model, prefix='' if hasattr(model, 'gpt2') else 'gpt2.')  
        if len(missing_keys) > 0:  
            logger.info("Weights of {} not initialized from pretrained model: {}".format(  
                model.__class__.__name__, missing_keys))  
        if len(unexpected_keys) > 0:  
            logger.info("Weights from pretrained model not used in {}: {}".format(  
                model.__class__.__name__, unexpected_keys))  
        return model  

BERT(PretrainedBertModel) --> GPT2WithEmbeddings

In [18]:
class GPT2TransformerBlock(nn.Module):  
    def __init__(self, config: GPT2Config):  
        super().__init__()  
        gpt2_model = GPT2Model(config)  
        self.block = gpt2_model.h[0]  # Use the first block from the GPT2 model  
  
    def forward(self, x, layer_past=None, attention_mask=None, head_mask=None):  
        x = self.block(x, layer_past=layer_past, attention_mask=attention_mask, head_mask=head_mask)  
        return x[0]  # Return only the hidden states, discard 'presents' 

In [19]:
class GPT2WithEmbeddings(PreTrainedGPT2Model):  
    """  
    GPT2 model with custom embeddings.  
    """  
  
    def __init__(self, config: GPT2Config, dx_voc=None, rx_voc=None):  
        """  
        :param config: GPT2Config instance  
        :param dx_voc: Vocabulary for diagnosis codes  
        :param rx_voc: Vocabulary for prescription codes  
        """  
        super().__init__(config)  
        if config.graph:  
            assert dx_voc is not None  
            assert rx_voc is not None  
  
        # Replace the default GPT2 embeddings with custom FuseEmbeddings  
        #self.gpt2 = CustomGPT2Model(config, dx_voc, rx_voc)  
        self.gpt2 = FuseEmbeddings(config, dx_voc, rx_voc) 
        # Multi-layers transformer blocks  
        self.transformer_blocks = nn.ModuleList(  
            # [GPT2TransformerBlock(config) for _ in range(config.num_hidden_layers)]) 
            [GPT2TransformerBlock(config) for _ in range(config.n_layer)])  
  
        self.apply(self.init_gpt2_weights)  
  
    def forward(self, x, token_type_ids=None, input_positions=None, input_sides=None):  
        # Attention masking for padded token  
        # torch.ByteTensor([batch_size, 1, seq_len, seq_len)  
        mask = (x > 1).unsqueeze(1).repeat(1, x.size(1), 1).unsqueeze(1)  
  
        # Embedding the indexed sequence to sequence of vectors  
        x = self.gpt2.embeddings(x, token_type_ids)  
  
        # Running over multiple transformer blocks  
        for transformer in self.transformer_blocks:  
            x = transformer.forward(x, mask)  
  
        return x, x[:, 0]  

GBERT_Pretrain --> GGPT2_Pretrain

In [20]:
# The MappingHead class is a simple feed-forward neural network 
# with a single dense layer followed by a ReLU activation. 
#It's not specifically tied to BERT or GPT2 and can be used with any 
#architecture, as long as the input and output dimensions match.  

class MappingHead(nn.Module):  
    def __init__(self, config: GPT2Config):  
        super(MappingHead, self).__init__()  
        # self.dense = nn.Sequential(nn.Linear(config.hidden_size, config.hidden_size),  
        #                            nn.ReLU())  
        self.dense = nn.Sequential(nn.Linear(config.n_embd, config.n_embd),  
                                   nn.ReLU())  
  
    def forward(self, input):  
        return self.dense(input)  

In [21]:
class ClsHead(nn.Module):
    def __init__(self, config: GPT2Config, voc_size):
        super(ClsHead, self).__init__()
        self.cls = nn.Sequential(nn.Linear(config.n_embd, config.n_embd), nn.ReLU(
        ), nn.Linear(config.n_embd, voc_size))

    def forward(self, input):
        return self.cls(input)

In [22]:
class SelfSupervisedHead(nn.Module):  
    def __init__(self, config: GPT2Config, dx_voc_size, rx_voc_size):  
        super(SelfSupervisedHead, self).__init__()  
        self.multi_cls = nn.ModuleList([ClsHead(config, dx_voc_size), ClsHead(  
            config, dx_voc_size), ClsHead(config, rx_voc_size), ClsHead(config, rx_voc_size)])  
  
    def forward(self, dx_inputs, rx_inputs):  
        # inputs (B, hidden)  
        # output logits  
        return self.multi_cls[0](dx_inputs), self.multi_cls[1](rx_inputs), self.multi_cls[2](dx_inputs), self.multi_cls[3](rx_inputs)  


In [23]:
#pre training
from transformers import GPT2Model, GPT2Config  
import torch.nn as nn  
  
class CustomGPT2Model(GPT2Model):  
    def __init__(self, config):  
        super(CustomGPT2Model, self).__init__(config)  
  
class GGPT2_Pretrain(nn.Module):  
    def __init__(self, config: GPT2Config, dx_voc=None, rx_voc=None):  
        super(GGPT2_Pretrain, self).__init__()  
        self.dx_voc_size = len(dx_voc.word2idx)  
        self.rx_voc_size = len(rx_voc.word2idx)  
  
        self.gpt2 = GPT2WithEmbeddings(config)  
        self.cls = SelfSupervisedHead(  
            config, self.dx_voc_size, self.rx_voc_size)  
    def forward(self, inputs, dx_labels=None, rx_labels=None):  
        # inputs (B, 2, max_len)  
        # gpt2_pool (B, hidden)  
        #print("Inputs shape:", inputs.shape)  # Add this line to print the shape of inputs  
  
        attention_mask = torch.zeros((inputs.size(0), inputs.size(2))).long().to(inputs.device)  
        dx_gpt2_outputs = self.gpt2(inputs[:, 0, :], attention_mask=attention_mask)  
        dx_gpt2_pool = dx_gpt2_outputs.last_hidden_state[:, 0, :]  
  
        rx_gpt2_outputs = self.gpt2(inputs[:, 1, :], attention_mask=attention_mask)  
        rx_gpt2_pool = rx_gpt2_outputs.last_hidden_state[:, 0, :]  
  
        dx2dx, rx2dx, dx2rx, rx2rx = self.cls(dx_gpt2_pool, rx_gpt2_pool)  
        # output logits  
        if rx_labels is None or dx_labels is None:  
            return F.sigmoid(dx2dx), F.sigmoid(rx2dx), F.sigmoid(dx2rx), F.sigmoid(rx2rx)  
        else:  
            loss = F.binary_cross_entropy_with_logits(dx2dx, dx_labels) + \
                F.binary_cross_entropy_with_logits(rx2dx, dx_labels) + \
                F.binary_cross_entropy_with_logits(dx2rx, rx_labels) + \
                F.binary_cross_entropy_with_logits(rx2rx, rx_labels)  
            return loss, F.sigmoid(dx2dx), F.sigmoid(rx2dx), F.sigmoid(dx2rx), F.sigmoid(rx2rx) 
  

Replacement of Run_pretraining.py

In [24]:
# Voc class is a simple vocabulary class that maps words to indices and vice versa. It's not specifically tied to BERT or GPT2 and can be used with any architecture

class Voc(object):
    def __init__(self):
        self.idx2word = {}
        self.word2idx = {}

    def add_sentence(self, sentence):
        for word in sentence:
            if word not in self.word2idx:
                self.idx2word[len(self.word2idx)] = word
                self.word2idx[word] = len(self.word2idx)

In [25]:
import random
def random_word(tokens, vocab):
    for i, _ in enumerate(tokens):
        prob = random.random()
        # mask token with 15% probability
        if prob < 0.15:
            prob /= 0.15

            # 80% randomly change token to mask  --> EOS token
            if prob < 0.8:
                tokens[i] = "[EOS]"
            # 10% randomly change token to random token
            elif prob < 0.9:
                tokens[i] = random.choice(list(vocab.word2idx.items()))[0]
            else:
                pass
        else:
            pass

    return tokens

For GPT2, the main difference is that it does not use the "[MASK]" token since it is not trained with masked language modeling. GPT2 is an autoregressive model, which predicts the next token in the sequence given the previous tokens. However, we can still use the "[PAD]" and "[CLS]" tokens if needed for padding and classification purposes.

GPT2 uses the "[EOS]" (end of sentence) token, which can be added for testing purpose.

In [26]:
# Pretraining code 
class EHRTokenizer(object):
    """Runs end-to-end tokenization"""

    def __init__(self, data_dir, special_tokens=("[PAD]", "[CLS]", "[EOS]")):

        self.vocab = Voc()

        # special tokens
        self.vocab.add_sentence(special_tokens)

        self.rx_voc = self.add_vocab(os.path.join(data_dir, 'rx-vocab.txt'))
        self.dx_voc = self.add_vocab(os.path.join(data_dir, 'dx-vocab.txt'))

    def add_vocab(self, vocab_file):
        voc = self.vocab
        specific_voc = Voc()
        with open(vocab_file, 'r') as fin:
            for code in fin:
                voc.add_sentence([code.rstrip('\n')])
                specific_voc.add_sentence([code.rstrip('\n')])
        return specific_voc

    def convert_tokens_to_ids(self, tokens):
        """Converts a sequence of tokens into ids using the vocab."""
        ids = []
        for token in tokens:
            ids.append(self.vocab.word2idx[token])
        return ids

    def convert_ids_to_tokens(self, ids):
        """Converts a sequence of ids in wordpiece tokens using the vocab."""
        tokens = []
        for i in ids:
            tokens.append(self.vocab.idx2word[i])
        return tokens

In [27]:
# Pretraining code 
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset
class EHRDataset(Dataset):
    def __init__(self, data_pd, tokenizer: EHRTokenizer, max_seq_len=55):
        self.data_pd = data_pd
        self.tokenizer = tokenizer
        self.seq_len = max_seq_len
        # print(max_seq_len)

        self.sample_counter = 0

        def transform_data(data):
            """
            :param data: raw data form
            :return: {subject_id, [adm, 2, codes]},
            """
            admissions = []
            for _, row in data.iterrows():
                admission = [list(row['ICD9_CODE']), list(row['ATC4'])]
                admissions.append(admission)
            return admissions

        self.admissions = transform_data(data_pd)

    def __len__(self):
        return len(self.admissions)

    def __getitem__(self, item):
        cur_id = item
        adm = copy.deepcopy(self.admissions[item])

        def fill_to_max(l, seq):
            while len(l) < seq:
                l.append('[PAD]')
            return l
        """y
        """
        y_dx = np.zeros(len(self.tokenizer.dx_voc.word2idx))
        y_rx = np.zeros(len(self.tokenizer.rx_voc.word2idx))
        for item in adm[0]:
            y_dx[self.tokenizer.dx_voc.word2idx[item]] = 1
        for item in adm[1]:
            y_rx[self.tokenizer.rx_voc.word2idx[item]] = 1

        """replace tokens with [EOS]
        """
        adm[0] = random_word(adm[0], self.tokenizer.rx_voc)
        adm[1] = random_word(adm[1], self.tokenizer.dx_voc)

        """extract input and output tokens
        """
        random_word
        input_tokens = []  # (2*max_len)
        input_tokens.extend(
            ['[CLS]'] + fill_to_max(list(adm[0]), self.seq_len - 1))
        input_tokens.extend(
            ['[CLS]'] + fill_to_max(list(adm[1]), self.seq_len - 1))

        """convert tokens to id
        """
        input_ids = self.tokenizer.convert_tokens_to_ids(input_tokens)
        #print("what is input id",input_ids)

        if cur_id < 5:
            print("*** Example ***")
            print("input tokens: %s" % " ".join(
                [str(x) for x in input_tokens]))
            print("input_ids: %s" %
                        " ".join([str(x) for x in input_ids]))
        #print(torch.tensor(input_ids, dtype=torch.long))
        cur_tensors = (torch.tensor(input_ids, dtype=torch.long).view(-1, self.seq_len),
                       torch.tensor(y_dx, dtype=torch.float),
                       torch.tensor(y_rx, dtype=torch.float))

        return cur_tensors

In [28]:
# Pretraining code 
def load_dataset(data_dir ,max_seq_length):
    # data_dir = data_dir
    # max_seq_len = max_seq_length

    # load tokenizer
    print("check in load dataset",max_seq_length)
    tokenizer = EHRTokenizer(data_dir)

    # load data
    data_multi = pd.read_pickle(os.path.join(
        data_dir, 'data-multi-visit.pkl')).iloc[:, :4]
    data_single = pd.read_pickle(
        os.path.join(data_dir, 'data-single-visit.pkl'))

    # load trian, eval, test data
    ids_file = [os.path.join(data_dir, 'train-id.txt'),
                os.path.join(data_dir, 'eval-id.txt'),
                os.path.join(data_dir, 'test-id.txt')]

    def load_ids(data, file_name):
        """
        :param data: multi-visit data
        :param file_name:
        :return: raw data form
        """
        ids = []
        with open(file_name, 'r') as f:
            for line in f:
                ids.append(int(line.rstrip('\n')))
        return data[data['SUBJECT_ID'].isin(ids)].reset_index(drop=True)
    # return tokenizer, \
    #     EHRDataset(load_ids(data_multi, ids_file[1]), tokenizer, max_seq_len), \
    #     EHRDataset(load_ids(data_multi, ids_file[1]), tokenizer, max_seq_len), \
    #     EHRDataset(load_ids(data_multi, ids_file[2]), tokenizer, max_seq_len)
    return tokenizer, \
        EHRDataset(pd.concat([data_single, load_ids(
            data_multi, ids_file[0])]), tokenizer, max_seq_length), \
        EHRDataset(load_ids(data_multi, ids_file[1]), tokenizer, max_seq_length), \
        EHRDataset(load_ids(data_multi, ids_file[2]), tokenizer, max_seq_length)

In [29]:
# Pretraining code 
# Update below param like do_train etc and control the if loop for training
model_name = "GGPT-pretraining"
data_dir = "../data"
pretrain_dir = "../saved/GBert-pretraining"
train_file = "data-multi-visit.pkl"
output_dir = "../saved/"
use_pretrain = False
graph = False
therhold = 0.3
max_seq_length = 55
do_train = True
do_eval = True
do_test = True
train_batch_size = 1
learning_rate = 5e-4
num_train_epochs = 10
seed = 1203
warmup_proportion = 0.1
batch_size = 16
#64

In [30]:
# Pretraining code 
output_dir = os.path.join(output_dir, model_name)

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [22]:
# Pretraining code 
if not do_train and not do_eval:
        raise ValueError(
            "At least one of `do_train` or `do_eval` must be True.")

In [23]:
# Pretraining code 
os.makedirs(output_dir, exist_ok=True)

In [24]:
# Pretraining code 
print("Loading Dataset")
tokenizer, train_dataset, eval_dataset, test_dataset = load_dataset(data_dir ,max_seq_length=55)
train_dataloader = DataLoader(train_dataset,
                                  sampler=RandomSampler(train_dataset),
                                  batch_size=batch_size)
eval_dataloader = DataLoader(eval_dataset,
                                 sampler=SequentialSampler(eval_dataset),
                                 batch_size=batch_size)
test_dataloader = DataLoader(test_dataset,
                                 sampler=SequentialSampler(test_dataset),
                                 batch_size=batch_size)
print("Loading Dataset Complete")

Loading Dataset
check in load dataset 55
Loading Dataset Complete


In [25]:
# Pretraining code     
from utils import metric_report, t2n, get_n_params

print('Loading Model: ' + model_name)
if use_pretrain:
    print("Use Pretraining model")
    model = GGPT2_Pretrain.from_pretrained(pretrain_dir, dx_voc=tokenizer.dx_voc,
                                           rx_voc=tokenizer.rx_voc)
    # GBERT_Pretrain.from_pretrained(args.pretrain_dir, dx_voc=tokenizer.dx_voc,
    #                                            rx_voc=tokenizer.rx_voc)
else:
    # config = BertConfig(
    #     vocab_size_or_config_json_file=len(tokenizer.vocab.word2idx))
    # config.graph = args.graph
    # model = GBERT_Pretrain(config, tokenizer.dx_voc, tokenizer.rx_voc)
    # logger.info('# of model parameters: ' + str(get_n_params(model)))
    print("else")
    config = GPT2Config(vocab_size_or_config_json_file=len(tokenizer.vocab.word2idx))
    config.graph = graph
    model = GGPT2_Pretrain(config, tokenizer.dx_voc, tokenizer.rx_voc)
    print('# of model parameters: ' + str(get_n_params(model)))

    model.to(device)

    model_to_save = model.module if hasattr(
        model, 'module') else model  # Only save the model it-self
    dx_output_model_file = os.path.join(output_dir, "pytorch_model_gptpre.bin")

Loading Model: GGPT-pretraining
else
# of model parameters: 130593346


In [26]:
optimizer = Adam(model.parameters(), lr=learning_rate)

In [30]:
#THIS WILL TAKE 8.30 Hours to Finish on NV6 TESLA M60 GPU
if do_train:
    #writer = SummaryWriter(args.output_dir)

    print("***** Running training *****")
    print("  Num examples = %d", len(train_dataset))
    print("  Batch size = %d", 1)

    dx_acc_best, rx_acc_best = 0, 0
    acc_name = 'prauc'

    global_step = 0
    for _ in trange(int(num_train_epochs), desc="Epoch"):
        print('')
        #print('for loop for epoch')
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        prog_iter = tqdm(train_dataloader, leave=False, desc='Training')
        #print("length of prog iter ",len(prog_iter))
        model.train()
        for _, batch in enumerate(prog_iter):
            batch = tuple(t.to(device) for t in batch)
            input_ids, dx_labels, rx_labels = batch
            loss, dx2dx, rx2dx, dx2rx, rx2rx = model(
                input_ids, dx_labels, rx_labels)
                # input_ids, dx_labels, rx_labels = input_ids.squeeze(
                # ), dx_labels.squeeze(), rx_labels.squeeze(dim=0)
                # loss, dx_logits, rx_logits = model(input_ids, dx_labels=dx_labels, rx_labels=rx_labels,
                #                                    epoch=global_step)
            loss.backward()

            tr_loss += loss.item()
            nb_tr_examples += 1
            nb_tr_steps += 1

            # Display loss
            prog_iter.set_postfix(loss='%.4f' % (tr_loss / nb_tr_steps))

            optimizer.step()
            optimizer.zero_grad()

        print('train/loss', tr_loss / nb_tr_steps, global_step)
        global_step += 1

        if do_eval:
            print('')
            print("***** Running eval *****")
            model.eval()
            dx2dx_y_preds = []
            rx2dx_y_preds = []
            dx_y_trues = []

            dx2rx_y_preds = []
            rx2rx_y_preds = []
            rx_y_trues = []
            for batch in tqdm(eval_dataloader, desc="Evaluating"):
                batch = tuple(t.to(device) for t in batch)
                input_ids, dx_labels, rx_labels = batch
                with torch.no_grad():
                    dx2dx, rx2dx, dx2rx, rx2rx = model(input_ids)
                    dx2dx_y_preds.append(t2n(dx2dx))
                    rx2dx_y_preds.append(t2n(rx2dx))
                    dx2rx_y_preds.append(t2n(dx2rx))
                    rx2rx_y_preds.append(t2n(rx2rx))

                    dx_y_trues.append(
                        t2n(dx_labels))
                    rx_y_trues.append(
                        t2n(rx_labels))

            print('')
            print('dx2dx')
            dx2dx_acc_container = metric_report(
                np.concatenate(dx2dx_y_preds, axis=0), np.concatenate(dx_y_trues, axis=0), therhold)
            print('rx2dx')
            rx2dx_acc_container = metric_report(
                np.concatenate(rx2dx_y_preds, axis=0), np.concatenate(dx_y_trues, axis=0), therhold)
            print('dx2rx')
            dx2rx_acc_container = metric_report(
                    np.concatenate(dx2rx_y_preds, axis=0), np.concatenate(rx_y_trues, axis=0), therhold)
            print('rx2rx')
            rx2rx_acc_container = metric_report(
                    np.concatenate(rx2rx_y_preds, axis=0), np.concatenate(rx_y_trues, axis=0), therhold)

            # keep in history
            for k, v in dx2dx_acc_container.items():
                print(
                        'eval_dx2dx/{}'.format(k), v, global_step)
            for k, v in rx2dx_acc_container.items():
                print(
                        'eval_rx2dx/{}'.format(k), v, global_step)
            for k, v in dx2rx_acc_container.items():
                print(
                        'eval_dx2rx/{}'.format(k), v, global_step)
            for k, v in rx2rx_acc_container.items():
                print(
                        'eval_rx2rx/{}'.format(k), v, global_step)

            if rx2rx_acc_container[acc_name] > dx_acc_best:
                dx_acc_best = rx2rx_acc_container[acc_name]
                    # save model
                torch.save(model_to_save.state_dict(),
                               dx_output_model_file)

                #with open(os.path.join(output_dir, 'ggpt2_config.json'), 'w', encoding='utf-8') as fout:
                        #fout.write(model.config.to_json_string())


***** Running training *****
  Num examples = %d 41220
  Batch size = %d 1

*** Example ***
input tokens: [CLS] 431 5070 4280 [EOS] 2765 4019 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [CLS] B05CX A12CA J01MA A01AB C02DD [EOS] A02BC C02DB A12AA [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
input_ids: 1 2295 1622 2273 2 1377 1658 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 136 131 252 450 298 2 390 277 462 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

Training: 100%|██████████| 2577/2577 [24:15<00:00,  2.15it/s, loss=0.2330]
                                                                          
Evaluating: 100%|██████████| 142/142 [00:24<00:00,  5.76it/s]
04/25/2023 02:29:00 - INFO - utils -   jaccard    : 0.0323    
04/25/2023 02:29:00 - INFO - utils -   f1         : 0.0580    
04/25/2023 02:29:00 - INFO - utils -   prauc      : 0.1629    
04/25/2023 02:29:09 - INFO - utils -   jaccard    : 0.0323    
04/25/2023 02:29:09 - INFO - utils -   f1         : 0.0580    
04/25/2023 02:29:09 - INFO - utils -   prauc      : 0.1628    
04/25/2023 02:29:15 - INFO - utils -   jaccard    : 0.0000    
04/25/2023 02:29:15 - INFO - utils -   f1         : 0.0000    
04/25/2023 02:29:15 - INFO - utils -   prauc      : 0.1366    
04/25/2023 02:29:21 - INFO - utils -   jaccard    : 0.0000    
04/25/2023 02:29:21 - INFO - utils -   f1         : 0.0000    
04/25/2023 02:29:21 - INFO - utils -   prauc      : 0.1356    
Training:   3%|▎         | 74/25

_______________________________________________________________________


The above concluded the pre training
Now we start with adapting Runbert.py code


GGPT_PREDICT is pending

In [1]:
class EHRTokenizerPred(object):  
    """Runs end-to-end tokenization"""  
  
    def __init__(self, data_dir, special_tokens=("[PAD]", "[CLS]", "[EOS]")):  
  
        self.vocab = Voc()  
  
        # special tokens  
        self.vocab.add_sentence(special_tokens)  
  
        self.rx_voc = self.add_vocab(os.path.join(data_dir, 'rx-vocab.txt'))  
        self.dx_voc = self.add_vocab(os.path.join(data_dir, 'dx-vocab.txt'))  
  
        # code only in multi-visit data  
        self.rx_voc_multi = Voc()  
        self.dx_voc_multi = Voc()  
        with open(os.path.join(data_dir, 'rx-vocab-multi.txt'), 'r') as fin:  
            for code in fin:  
                self.rx_voc_multi.add_sentence([code.rstrip('\n')])  
        with open(os.path.join(data_dir, 'dx-vocab-multi.txt'), 'r') as fin:  
            for code in fin:  
                self.dx_voc_multi.add_sentence([code.rstrip('\n')])  
  
    def add_vocab(self, vocab_file):  
        voc = self.vocab  
        specific_voc = Voc()  
        with open(vocab_file, 'r') as fin:  
            for code in fin:  
                voc.add_sentence([code.rstrip('\n')])  
                specific_voc.add_sentence([code.rstrip('\n')])  
        return specific_voc  
  
    def convert_tokens_to_ids(self, tokens):  
        """Converts a sequence of tokens into ids using the vocab."""  
        ids = []  
        for token in tokens:  
            ids.append(self.vocab.word2idx[token])  
        return ids  
  
    def convert_ids_to_tokens(self, ids):  
        """Converts a sequence of ids in wordpiece tokens using the vocab."""  
        tokens = []  
        for i in ids:  
            tokens.append(self.vocab.idx2word[i])  
        return tokens

In [4]:
class EHRDatasetPred(Dataset):
    def __init__(self, data_pd, tokenizer: EHRTokenizerPred, max_seq_len):
        self.data_pd = data_pd
        self.tokenizer = tokenizer
        self.seq_len = max_seq_len

        self.sample_counter = 0

        def transform_data(data):
            """
            :param data: raw data form
            :return: {subject_id, [adm, 2, codes]},
            """
            records = {}
            for subject_id in data['SUBJECT_ID'].unique():
                item_df = data[data['SUBJECT_ID'] == subject_id]
                patient = []
                for _, row in item_df.iterrows():
                    admission = [list(row['ICD9_CODE']), list(row['ATC4'])]
                    patient.append(admission)
                if len(patient) < 2:
                    continue
                records[subject_id] = patient
            return records

        self.records = transform_data(data_pd)

    def __len__(self):
        return len(self.records)

    def __getitem__(self, item):
        cur_id = self.sample_counter
        self.sample_counter += 1
        subject_id = list(self.records.keys())[item]

        def fill_to_max(l, seq):
            while len(l) < seq:
                l.append('[PAD]')
            return l

        """extract input and output tokens
        """
        input_tokens = []  # (2*max_len*adm)
        output_dx_tokens = []  # (adm-1, l)
        output_rx_tokens = []  # (adm-1, l)

        # for idx, adm in enumerate(self.records[subject_id]):
        #     input_tokens.extend(
        #         ['[CLS]'] + fill_to_max(list(adm[0]), self.seq_len - 1))
        #     input_tokens.extend(
        #         ['[CLS]'] + fill_to_max(list(adm[1]), self.seq_len - 1))
        #     # output_rx_tokens.append(list(adm[1]))

        #     if idx != 0:
        #         output_rx_tokens.append(list(adm[1]))
        #         output_dx_tokens.append(list(adm[0]))
        for idx, adm in enumerate(self.records[subject_id]):  
            input_tokens.extend(  
                ['[CLS]'] + fill_to_max(list(adm[0]), self.seq_len - 2) + ['[EOS]'])  
            input_tokens.extend(  
            ['[CLS]'] + fill_to_max(list(adm[1]), self.seq_len - 2) + ['[EOS]'])  
  
            if idx != 0:  
                output_rx_tokens.append(list(adm[1]))  
                output_dx_tokens.append(list(adm[0]))  


        """convert tokens to id
        """
        input_ids = self.tokenizer.convert_tokens_to_ids(input_tokens)
        output_dx_labels = []  # (adm-1, dx_voc_size)
        output_rx_labels = []  # (adm-1, rx_voc_size)

        dx_voc_size = len(self.tokenizer.dx_voc_multi.word2idx)
        rx_voc_size = len(self.tokenizer.rx_voc_multi.word2idx)
        for tokens in output_dx_tokens:
            tmp_labels = np.zeros(dx_voc_size)
            tmp_labels[list(
                map(lambda x: self.tokenizer.dx_voc_multi.word2idx[x], tokens))] = 1
            output_dx_labels.append(tmp_labels)

        for tokens in output_rx_tokens:
            tmp_labels = np.zeros(rx_voc_size)
            tmp_labels[list(
                map(lambda x: self.tokenizer.rx_voc_multi.word2idx[x], tokens))] = 1
            output_rx_labels.append(tmp_labels)

        if cur_id < 5:
            print("*** Example ***")
            print("subject_id: %s" % subject_id)
            print("input tokens: %s" % " ".join(
                [str(x) for x in input_tokens]))
            print("input_ids: %s" %
                        " ".join([str(x) for x in input_ids]))

        assert len(input_ids) == (self.seq_len *
                                  2 * len(self.records[subject_id]))
        assert len(output_dx_labels) == (len(self.records[subject_id]) - 1)
        # assert len(output_rx_labels) == len(self.records[subject_id])-1

        cur_tensors = (torch.tensor(input_ids).view(-1, self.seq_len),
                       torch.tensor(output_dx_labels, dtype=torch.float),
                       torch.tensor(output_rx_labels, dtype=torch.float))

        return cur_tensors

In [5]:
def load_datasetpred(data_dir , max_seq_length):
    # data_dir = args.data_dir
    # max_seq_len = args.max_seq_length

    # load tokenizer
    tokenizer = EHRTokenizerPred(data_dir)

    # load data
    data = pd.read_pickle(os.path.join(data_dir, 'data-multi-visit.pkl'))

    # load trian, eval, test data
    ids_file = [os.path.join(data_dir, 'train-id.txt'),
                os.path.join(data_dir, 'eval-id.txt'),
                os.path.join(data_dir, 'test-id.txt')]

    def load_ids(data, file_name):
        """
        :param data: multi-visit data
        :param file_name:
        :return: raw data form
        """
        ids = []
        with open(file_name, 'r') as f:
            for line in f:
                ids.append(int(line.rstrip('\n')))
        return data[data['SUBJECT_ID'].isin(ids)].reset_index(drop=True)

    return tokenizer, tuple(map(lambda x: EHRDataset(load_ids(data, x), tokenizer, max_seq_length), ids_file))


In [6]:
# parser params converted to normal variables
model_name = "G_GPT-predict"
data_dir = "../data"
pretrain_dir = "../saved/GBert-pretraining"
train_file = "data-multi-visit.pkl"
output_dir = "../saved/"
use_pretrain = False
graph = False
therhold = 0.3
max_seq_length = 55
do_train = False
do_eval = True
do_test = True
train_batch_size = 1
learning_rate = 5e-4
num_train_epochs = 20
seed = 1203
warmup_proportion = 0.1

In [8]:
import random

In [9]:
output_dir = os.path.join(output_dir,model_name)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if not do_train and not do_eval:
        raise ValueError(
            "At least one of `do_train` or `do_eval` must be True.")
os.makedirs(output_dir, exist_ok=True)

In [10]:
print("Loading Dataset")
tokenizer, (train_dataset, eval_dataset, test_dataset) = load_datasetpred(data_dir , max_seq_length)

Loading Dataset


NameError: name 'Voc' is not defined